In [1]:
import tensorflow as tf
from pilco.policies.rbf_policy import RBFPolicy

In [22]:
rbf_policy = RBFPolicy(2, 1, 5, dtype=tf.float32)
rbf_policy.reset()

In [23]:
loc = tf.ones(2, dtype=tf.float32)
cov = tf.eye(2, dtype=tf.float32)

mean_u, cov_su, cov_uu = rbf_policy.match_moments(loc, cov)

In [24]:
mean_u

<tf.Tensor: shape=(), dtype=float32, numpy=0.94104517>

In [25]:
cov_su

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-0.571962  , -0.11883277], dtype=float32)>

In [26]:
cov_uu

<tf.Tensor: shape=(), dtype=float32, numpy=-0.0794726>

In [27]:
cov1 = tf.concat([cov, cov_su[None, :]], axis=0)
cov2 = tf.concat([cov_su[:, None], cov_uu[None, None]], axis=0)

cov_full = tf.concat([cov1, cov2], axis=1)

tf.linalg.eig(cov_full)[0]

<tf.Tensor: shape=(3,), dtype=complex64, numpy=array([-0.33508354+0.j,  0.99999994-0.j,  1.2556115 +0.j], dtype=complex64)>

In [28]:
cov_full

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[ 1.        ,  0.        , -0.571962  ],
       [ 0.        ,  1.        , -0.11883277],
       [-0.571962  , -0.11883277, -0.0794726 ]], dtype=float32)>